# Generating NN Code Bench OOB Extension

This is an experimental process to expand the smaller sized samples of NN Code Bench for the purpose of creating a dataset that can be used to train NN models. The samples that are going to be used to expand the dataset, through fuzzing techniques, are the **Polynomial Approximation Networks**, the **Hopfield Nets**, the **Reach Prob Density**, and the **Reinforcement Learning** samples. The **Activation Functions** and the **Math Functions** will not be used as they are too small or don't have any code that can cause memory leaks if manipulated (Dont use math and activation categories as they dont contain array out of bounds (They are too small)).

The code extensions will focus on array out of bounds, as it is an easy safety property to detect.

* Timeout: 900s as in SVComp - 30s for testing.

The following research questions are going to be answered by the expansion of the dataset:

* What size can ESBMC handle?
* Will ESBMC be able to find the bug in a reasonable time? Compare 30s vs 900s.
* How do other software versifiers perform with the task of solving the expanded dataset?

## TODO

- Remove assert safety properties by Eduardo in the code.


**Note on Mutation Testing**: We are interested only in the killed mutations as those indicate that the output of the program has changed.


## Preprocess Data

This notebook takes care of processing the data of NN Codebench by compiling them into executables. Takes the programs in `networks` and compiles them to `bin` in the same structure.

### Base Dataset

#### Building Base Source Dataset

Building it from scratch.

In [1]:
import os
import shutil
from contextlib import chdir
from typing import Tuple, Optional
from subprocess import Popen, STDOUT, PIPE

In [2]:
def popen(cmd: str) -> Tuple[str, str]:
    process = Popen(cmd, stdout=PIPE, stderr=STDOUT)
    (output_bytes, err_bytes) = process.communicate()
    # Return
    exit_code = process.wait()
    output: str = str(output_bytes).replace("\\n", "\n")
    err: str = str(err_bytes).replace("\\n", "\n")
    return output, err

In [3]:
BUILD_PATH: str = "./build"
build_folders: list[str] = [
    f"{BUILD_PATH}/export/hopfield_nets",
    f"{BUILD_PATH}/export/poly_approx",
    f"{BUILD_PATH}/export/reach_prob_density",
    f"{BUILD_PATH}/export/reinforcement_learning",
]

In [3]:
# Create the build folder in case it does not exist.
if os.path.exists(BUILD_PATH):
    print(f"{BUILD_PATH} already exists.")
else:
    os.makedirs(BUILD_PATH)
    print(f"Created {BUILD_PATH}")

./build already exists.


In [4]:
# Build all base benchmarks. Note: From my experience, these do not build on an ARM machine but, can be
# transferred from an x86_64 based machine and used, since they are still C source code files and not
# binary.
with chdir(BUILD_PATH):
    print("Building all samples...")
    print(os.popen("cmake ..").read())
    print(os.popen("make -j4 install").read())
    print("Completed...")

Building all samples...
-- Configuring done
-- Generating done
-- Build files have been written to: /home/yiannis/git/plain_c_nn_benchmark/build

[  1%] Built target reach_prob_lib
[  1%] Built target stub
[  2%] Built target reinforcement_lib
[  3%] Built target keras_lib
[  3%] Producing elu_0_safe.c-amalgamation.i
[  3%] Producing elu_2_safe.c-amalgamation.i
[  3%] Producing elu_1_safe.c-amalgamation.i
[  4%] Producing elu_3_unsafe.c-amalgamation.i
[  4%] Built target elu_1_safe.c-amalgamation
[  4%] Built target elu_3_unsafe.c-amalgamation
[  4%] Built target elu_2_safe.c-amalgamation
[  4%] Built target elu_0_safe.c-amalgamation
[  4%] Producing gelu_2_safe.c-amalgamation.i
[  4%] Producing elu_4_safe.c-amalgamation.i
[  5%] Producing gelu_0_safe.c-amalgamation.i
[  5%] Producing gelu_1_safe.c-amalgamation.i
[  5%] Built target gelu_2_safe.c-amalgamation
[  5%] Built target elu_4_safe.c-amalgamation
[  5%] Built target gelu_0_safe.c-amalgamation
[  5%] Built target gelu_1_safe.c-a

#### Remove intermediate files (speed up processing).

In [5]:
if os.path.exists(BUILD_PATH):
    count: int = 0
    with chdir(BUILD_PATH):
        files: list[str] = os.listdir(".")
        for file in files:
            if os.path.isfile(file) and file.endswith((".c", ".i", ".yml")):
                os.remove(file)
                count += 1
    print(f"Cleaned up {count} intermediate files.")

Cleaned up 1827 intermediate files.


#### Extracting it from SV-Bench

They mirror the built ones. So if we want to use the ones from SV-Bench, we can do so by
looking at the manually built files names (for example poly-approx) and transfering the
ones we want from the SV-Bench folder to the built one (replacing the manually built ones).

Why? Because of [this](https://github.com/emanino/plain_c_nn_benchmark/issues/24).

Create a `sv-bench` folder in the root of the project with the contents of `sv-bench/c/neural-networks`.

In [8]:
def transfer_from_sv_bench(dest_path: str, delete_source=False) -> None:
    """Transfers the files from SV Bench to the dest path. If `delete_source` is True
    then the transfer will also remove from the SV_Bench folder."""
    SV_BENCH_FOLDER = "sv-bench"
    print(f"Moving files from {SV_BENCH_FOLDER} to {dest_path}")
    # The folder for SV-Bench neural network samples.
    if not os.path.exists(SV_BENCH_FOLDER):
        raise FileNotFoundError(f"Directory SV-Bench does not exist: {SV_BENCH_FOLDER}")
    if not os.path.exists(dest_path):
        raise FileNotFoundError(f"Directory {dest_path} does not exist.")

    count: int = 0
    for idx, file in enumerate(os.listdir(dest_path)):
        src: str = f"{SV_BENCH_FOLDER}/{file}"
        dst: str = f"{dest_path}/{file}"
        if os.path.isfile(src) and file.endswith(".c"):
            shutil.copy(src=src, dst=dst)
            print(f"{idx}: {file}")
            count += 1

    print(f"Completed moving {count} files from {SV_BENCH_FOLDER} to {dest_path}\n")

# Copy SV-Bench files to each folder used.
for folder in build_folders:
    transfer_from_sv_bench(folder)

Moving files from sv-bench to ./build/export/hopfield_nets
2: tanh_w4_r1_case_0_unsafe.c-amalgamation.c
5: tanh_w4_r1_case_1_safe.c-amalgamation.c
8: tanh_w4_r2_case_0_safe.c-amalgamation.c
11: tanh_w4_r2_case_1_unsafe.c-amalgamation.c
14: tanh_w4_r3_case_0_safe.c-amalgamation.c
17: tanh_w4_r3_case_1_unsafe.c-amalgamation.c
20: tanh_w4_r4_case_0_safe.c-amalgamation.c
23: tanh_w4_r4_case_1_unsafe.c-amalgamation.c
26: tanh_w8_r1_case_0_unsafe.c-amalgamation.c
29: tanh_w8_r1_case_1_safe.c-amalgamation.c
32: tanh_w8_r2_case_0_safe.c-amalgamation.c
35: tanh_w8_r2_case_1_unsafe.c-amalgamation.c
38: tanh_w8_r3_case_0_safe.c-amalgamation.c
41: tanh_w8_r3_case_1_unsafe.c-amalgamation.c
44: tanh_w8_r4_case_0_safe.c-amalgamation.c
50: tanh_w16_r1_case_0_unsafe.c-amalgamation.c
53: tanh_w16_r1_case_1_safe.c-amalgamation.c
56: tanh_w16_r2_case_0_safe.c-amalgamation.c
59: tanh_w16_r2_case_1_unsafe.c-amalgamation.c
62: tanh_w16_r3_case_0_safe.c-amalgamation.c
65: tanh_w16_r3_case_1_safe.c-amalgamatio

### Prepare and Build Base Samples

We now have the base samples built. So we need to mutate and build base samples.

In [47]:
def build_sample(sample_file_path: str) -> None:
    # Mull compiler plugin parameters.
    MULL: str = "-fexperimental-new-pass-manager -fpass-plugin=/usr/lib/mull-ir-frontend-15 -g -grecord-command-line"

    # Check if sample file exists in path specified.
    if not os.path.exists(sample_file_path):
        raise FileExistsError(f"No file found in {sample_file_path}")

    # Get the base name.
    sample_name: str = os.path.basename(sample_file_path)

    # Check if it is a C source code file.
    if not sample_name.endswith(".c"):
        raise ValueError(f"{sample_name} is not a C source code filename.")

    # Check if it exists in the bin folder.
    if not os.path.exists("bin"):
        raise FileNotFoundError("No bin folder...")

    # Verifier stub include file.
    ver_h: str = "includes"
    # Add networks folder as include directory.
    poly_aprox_h: str = "networks"

    # Source files to compile and create binary.
    source_files: list[str] = ["includes/verifier_stubs.c", sample_file_path]
    
    # Intermediate source files, because Mull requires to build each file individually into an .o file.
    # Automatically built from source files list.
    object_files: list[str] = ["bin/intermediate/" + os.path.basename(source_file)[:-2] + ".o" for source_file in source_files]
    
    # Build each file into an object file (-c) and then combine. This is due to the Mull plugin,
    # which only works on single files.
    for source_file, object_file in zip(source_files, object_files):
        cmd: str = f"clang-15 {MULL} -c {source_file} -I{ver_h} -I{poly_aprox_h} -o {object_file}"
        print("\n" + cmd)
        print(os.popen(cmd).read())
    else:
        print()

    # The out directory is the dirname of the source file path placed inside the bin folder, in
    # order to preserve structure.
    out: str = f"./bin/{os.path.basename(os.path.dirname(sample_file_path))}/{sample_name.removesuffix('.c')}"
    
    # Link object files into single binary.
    cmd: str = f"clang-15 {MULL} {' '.join(object_files)} -I{ver_h} -I{poly_aprox_h} -lm -o {out}"
    print(cmd)
    print(os.popen(cmd).read())

    print(f"Compilation Complete: {out}")

In [48]:
# Create the bin folder in case it does not exist.
if os.path.exists("bin"):
    print("bin already exists.")
else:
    os.makedirs("bin")
    print("Created bin")

if os.path.exists("bin/intermediate"):
    print("bin/intermediate already exists.")
else:
    os.makedirs("bin/intermediate")
    print("Created bin/intermediate")

for folder in build_folders:
    folder_path: str = f"bin/{os.path.basename(folder)}"
    if os.path.exists(folder_path):
        print(folder_path, "already exists.")
    else:
        os.makedirs(folder_path)
        print("Created", folder_path)

Created bin
Created bin/intermediate
Created bin/hopfield_nets
Created bin/poly_approx
Created bin/reach_prob_density
Created bin/reinforcement_learning


In [49]:
count: int = 0
for folder in build_folders:
    print("Compiling folder:", folder)
    for file in os.listdir(folder):
        file_path: str = f"{folder}/{file}"
        if os.path.isfile(file_path) and file.endswith(".c"):
            print(f"Building {count} {file_path}")
            build_sample(file_path)
            print()
            count += 1
    print()

print(f"Compilation of samples has completed: Total {count}...")
del count

Compiling folder: ./build/export/hopfield_nets
Building 0 ./build/export/hopfield_nets/tanh_w4_r1_case_0_unsafe.c-amalgamation.c

clang-15 -fexperimental-new-pass-manager -fpass-plugin=/usr/lib/mull-ir-frontend-15 -g -grecord-command-line -c includes/verifier_stubs.c -Iincludes -Inetworks -o bin/intermediate/verifier_stubs.o
[info] Using configuration /home/yiannis/git/plain_c_nn_benchmark/mull.yml
[warning] Invalid regex for exclude-path: '^(?=.*\/(networks)\/).*$' has been ignored. Error: repetition-operator operand invalid
[info] Found compilation flags in the input bitcode
[info] Gathering functions under test (threads: 1)

       [################################] 1/1. Finished in 0ms
[info] Instruction selection (threads: 3)

       [################################] 3/3. Finished in 11ms
[info] Searching mutants across functions (threads: 3)

       [#####################-----------] 2/3
       [################################] 3/3. Finished in 10ms
[info] Applying filter: no d

./build/export/hopfield_nets/tanh_w8_r4_case_1_safe.c-amalgamation.c:9:62: error: expected ';' at end of declaration
    float input_array[32] = {0.0f}, output_array[32] = {0.0f}
                                                             ^
                                                             ;
./build/export/hopfield_nets/tanh_w8_r4_case_1_safe.c-amalgamation.c:10:62: error: expected ';' at end of declaration
        k2c_tensor input_tensor = {&input_array[0],2,32,{4,8,1,1,1}}
                                                                    ^
                                                                    ;
./build/export/hopfield_nets/tanh_w8_r4_case_1_safe.c-amalgamation.c:11:64: error: expected ';' at end of declaration
        k2c_tensor output_tensor = {&output_array[0],2,32,{4,8,1,1,1}}
                                                                      ^
                                                                      ;
./build/export/hopfield_nets/tanh_w

[info] Using configuration /home/yiannis/git/plain_c_nn_benchmark/mull.yml
[warning] Invalid regex for exclude-path: '^(?=.*\/(networks)\/).*$' has been ignored. Error: repetition-operator operand invalid
[info] Found compilation flags in the input bitcode
[info] Gathering functions under test (threads: 1)

       [################################] 1/1. Finished in 0ms
[info] Instruction selection (threads: 3)

       [################################] 3/3. Finished in 10ms
[info] Searching mutants across functions (threads: 3)

       [#####################-----------] 2/3
       [################################] 3/3. Finished in 11ms
[info] Applying filter: no debug info (threads: 1)

       [################################] 1/1. Finished in 0ms
[info] Applying filter: file path (threads: 1)

       [################################] 1/1. Finished in 0ms
[info] Applying filter: junk (threads: 1)

       [################################] 1/1. Finished in 11ms
[info] Prepare mutatio

./build/export/poly_approx/poly_1024_thresh_0_safe-checkpoint.c-amalgamation.c:10:60: error: expected ';' at end of declaration
    float input_array[1] = {0.0f}, output_array[1] = {0.0f}
                                                           ^
                                                           ;
./build/export/poly_approx/poly_1024_thresh_0_safe-checkpoint.c-amalgamation.c:11:64: error: expected ';' at end of declaration
    k2c_tensor input_tensor = {&input_array[0],1,1,{1,1,1,1,1}}
                                                               ^
                                                               ;
./build/export/poly_approx/poly_1024_thresh_0_safe-checkpoint.c-amalgamation.c:12:66: error: expected ';' at end of declaration
    k2c_tensor output_tensor = {&output_array[0],1,1,{1,1,1,1,1}}
                                                                 ^
                                                                 ;
./build/export/poly_approx/poly_1024_thr



clang-15 -fexperimental-new-pass-manager -fpass-plugin=/usr/lib/mull-ir-frontend-15 -g -grecord-command-line bin/intermediate/verifier_stubs.o bin/intermediate/poly_1024_thresh_0_safe-checkpoint.c-amalgamation.o -Iincludes -Inetworks -lm -o ./bin/poly_approx/poly_1024_thresh_0_safe-checkpoint.c-amalgamation

Compilation Complete: ./bin/poly_approx/poly_1024_thresh_0_safe-checkpoint.c-amalgamation

Building 99 ./build/export/poly_approx/poly_1024_thresh_0_safe.c-amalgamation.c

clang-15 -fexperimental-new-pass-manager -fpass-plugin=/usr/lib/mull-ir-frontend-15 -g -grecord-command-line -c includes/verifier_stubs.c -Iincludes -Inetworks -o bin/intermediate/verifier_stubs.o
[info] Using configuration /home/yiannis/git/plain_c_nn_benchmark/mull.yml
[warning] Invalid regex for exclude-path: '^(?=.*\/(networks)\/).*$' has been ignored. Error: repetition-operator operand invalid
[info] Found compilation flags in the input bitcode
[info] Gathering functions under test (threads: 1)

       [##

./build/export/poly_approx/poly_16_16_thresh_1_safe-checkpoint.c-amalgamation.c:10:60: error: expected ';' at end of declaration
    float input_array[1] = {0.0f}, output_array[1] = {0.0f}
                                                           ^
                                                           ;
./build/export/poly_approx/poly_16_16_thresh_1_safe-checkpoint.c-amalgamation.c:11:64: error: expected ';' at end of declaration
    k2c_tensor input_tensor = {&input_array[0],1,1,{1,1,1,1,1}}
                                                               ^
                                                               ;
./build/export/poly_approx/poly_16_16_thresh_1_safe-checkpoint.c-amalgamation.c:12:66: error: expected ';' at end of declaration
    k2c_tensor output_tensor = {&output_array[0],1,1,{1,1,1,1,1}}
                                                                 ^
                                                                 ;
./build/export/poly_approx/poly_16_16



clang-15 -fexperimental-new-pass-manager -fpass-plugin=/usr/lib/mull-ir-frontend-15 -g -grecord-command-line bin/intermediate/verifier_stubs.o bin/intermediate/poly_16_16_thresh_1_safe-checkpoint.c-amalgamation.o -Iincludes -Inetworks -lm -o ./bin/poly_approx/poly_16_16_thresh_1_safe-checkpoint.c-amalgamation

Compilation Complete: ./bin/poly_approx/poly_16_16_thresh_1_safe-checkpoint.c-amalgamation

Building 160 ./build/export/poly_approx/poly_16_16_thresh_0_safe.c-amalgamation.c

clang-15 -fexperimental-new-pass-manager -fpass-plugin=/usr/lib/mull-ir-frontend-15 -g -grecord-command-line -c includes/verifier_stubs.c -Iincludes -Inetworks -o bin/intermediate/verifier_stubs.o
[info] Using configuration /home/yiannis/git/plain_c_nn_benchmark/mull.yml
[warning] Invalid regex for exclude-path: '^(?=.*\/(networks)\/).*$' has been ignored. Error: repetition-operator operand invalid
[info] Found compilation flags in the input bitcode
[info] Gathering functions under test (threads: 1)

     

#### Delete Intermediate Build Files

In [53]:
if os.path.exists("./bin/intermediate/"):
    count: int = len(os.listdir("./bin/intermediate/"))
    shutil.rmtree("./bin/intermediate/")
    print(f"Cleaned up {count} /bin/intermediate/ intermediate files.")

### Generating Mutation Patches

Now that the mutations have been inserted into the compiled base samples, we have the mutation fuzzed samples. The mutation fuzzed samples can then be **executed** to produce the patches. The _killed_ patches are ones that cause the program execution to defer.

> #### WARNING! 
> Running the following section will execute code on the system. Make sure that the code is safe to run.

#### Delete Patch Files

In [6]:
if os.path.exists("./patches"):
    shutil.rmtree("./patches")
    print(f"Cleaned up ./patches intermediate files.")
    os.makedirs("./patches")

Cleaned up ./patches intermediate files.


In [20]:
# All the mutations that mull can apply. - This is not used. cxx_all is applied.
MUTATIONS: list[str] = [
    # "cxx_all",
    "cxx_add_assign_to_sub_assign",
    "cxx_add_to_sub",
    "cxx_and_assign_to_or_assign",
    "cxx_and_to_or",
    "cxx_assign_const",
    "cxx_bitwise_not_to_noop",
    "cxx_div_assign_to_mul_assign",
    "cxx_div_to_mul",
    "cxx_eq_to_ne",
    "cxx_ge_to_gt",
    "cxx_ge_to_lt",
    "cxx_gt_to_ge",
    "cxx_gt_to_le",
    "cxx_init_const",
    "cxx_le_to_gt",
    "cxx_le_to_lt",
    "cxx_logical_and_to_or",
    "cxx_logical_or_to_and",
    "cxx_lshift_assign_to_rshift_assign",
    "cxx_lshift_to_rshift",
    "cxx_lt_to_ge",
    "cxx_lt_to_le",
    "cxx_minus_to_noop",
    "cxx_mul_assign_to_div_assign",
    "cxx_mul_to_div",
    "cxx_ne_to_eq",
    "cxx_or_assign_to_and_assign",
    "cxx_or_to_and",
    "cxx_post_dec_to_post_inc",
    "cxx_post_inc_to_post_dec",
    "cxx_pre_dec_to_pre_inc",
    "cxx_pre_inc_to_pre_dec",
    "cxx_rem_assign_to_div_assign",
    "cxx_rem_to_div",
    "cxx_remove_negation",
    "cxx_remove_void_call",
    "cxx_replace_scalar_call",
    "cxx_rshift_assign_to_lshift_assign",
    "cxx_rshift_to_lshift",
    "cxx_sub_assign_to_add_assign",
    "cxx_sub_to_add",
    "cxx_xor_assign_to_or_assign",
    "cxx_xor_to_or",
    "negate_mutator",
    "scalar_value_mutator",
]

In [11]:
MULL_CONFIG_TEMPLATE = """mutators:
 - $operation
excludePaths: # support regex
 - ^(?=.*\/(networks)\/).*$
timeout: 10000 # milliseconds: 10 seconds
quiet: false # enables additional logging"""

In [15]:
def gen_mutation(sample_path: str, operation: Optional[str] = None, output_shell: bool=False) -> None:
    patch_name: str = os.path.basename(sample_path)
    patch_dir: str
    # Edit the config file to insert mutation setting.
    with open("mull.yml", "w") as file:
        if operation:
            patch_dir = f"patches/{operation}/{os.path.basename(os.path.dirname(sample_path))}"
            file.write(MULL_CONFIG_TEMPLATE.replace("$operation", operation))
        else:
            patch_dir = f"patches/{os.path.basename(os.path.dirname(sample_path))}"
            file.write(MULL_CONFIG_TEMPLATE.replace("$operation", "cxx_all"))
    # Allow surviving because will mark program as crash if survival.
    cmd = f"mull-runner-15 --reporters Patches --report-dir {patch_dir} --report-name {patch_name} --allow-surviving {sample_path}"
    print(cmd + "\n")
    
    p = Popen(cmd, shell=True, stderr=PIPE)
    exit_code: int = p.wait()
    if exit_code != 0:
        print(f"Error has occurred: {exit_code}")
        _, stderr = p.communicate()
        print(stderr)

In [21]:
# for mutation in MUTATIONS:
#     for build_folder in build_folders:
#         # Get bin folder from build folder.
#         bin_folder: str = f"./bin/{os.path.basename(build_folder)}"
#         print(f"Generating mutations for folder: {bin_folder}\n\n\n")
#         for file in os.listdir(bin_folder):
#             # Only do safe samples.
#             if "_safe" in file:
#                 print(f"Generating mutation: {file}\n")
#                 gen_mutation(f"{bin_folder}/{file}", mutation)
#             else:
#                 print(f"Skipping: {bin_folder}/{file}")
            
#             print()

for build_folder in build_folders:
    # Get bin folder from build folder.
    bin_folder: str = f"./bin/{os.path.basename(build_folder)}"
    print(f"Generating mutations for folder: {bin_folder}\n\n\n")
    for file in os.listdir(bin_folder):
        # Only do safe samples.
        if "_safe" in file:
            print(f"Generating mutation: {file}\n")
            gen_mutation(f"{bin_folder}/{file}", None)
        else:
            print(f"Skipping: {bin_folder}/{file}")
        
        print()

Generating mutations for folder: ./bin/hopfield_nets



Skipping: ./bin/hopfield_nets/tanh_w4_r1_case_0_unsafe.c-amalgamation

Generating mutation: tanh_w4_r1_case_1_safe.c-amalgamation

mull-runner-15 --reporters Patches --report-dir patches/hopfield_nets --report-name tanh_w4_r1_case_1_safe.c-amalgamation --allow-surviving ./bin/hopfield_nets/tanh_w4_r1_case_1_safe.c-amalgamation

[info] Using config /home/yiannis/git/plain_c_nn_benchmark/mull.yml
[warning] Could not find dynamic library: libm.so.6
[warning] Could not find dynamic library: libc.so.6
[warning] Invalid regex for exclude-path: '^(?=.*\/(networks)\/).*$' has been ignored. Error: repetition-operator operand invalid
[info] Warm up run (threads: 1)
       [################################] 1/1. Finished in 64ms
[info] Filter mutants (threads: 1)
       [################################] 1/1. Finished in 1ms
[info] Baseline run (threads: 1)
       [################################] 1/1. Finished in 47ms
[info] Running mutant

### Apply Killed Patches Only (patched-src)

In [4]:
if os.path.exists("./patched-src"):
    shutil.rmtree("./patched-src")
    print(f"Cleaned up ./patched-src files.")
os.makedirs("./patched-src")
print("Created ./patched-src directory.")
for folder in build_folders:
    new_dir: str = f"./patched-src/{os.path.basename(folder)}"
    os.makedirs(new_dir)
    print(f"Created {new_dir}")

Cleaned up ./patched-src files.
Created ./patched-src directory.
Created ./patched-src/hopfield_nets
Created ./patched-src/poly_approx
Created ./patched-src/reach_prob_density
Created ./patched-src/reinforcement_learning


In [5]:
def apply_patch(source_file_path: str, patch_path: str, result_path: str) -> None:
    cmd: str = f"patch -u {source_file_path} {patch_path} -o {result_path}"
    print(os.popen(cmd).read())

In [ ]:
# Iterate through build folders.
for build_folder in build_folders:
    sub_folder_name: str = os.path.basename(build_folder)
    # Iterate through source files (which are folders because mull has multiple patches
    # per folder).
    for file_patches_dir in os.listdir(f"patches/{sub_folder_name}"):
        # The patches are generated from the bin files which have the .c file extension removed.
        # So it needs to be added back to find the original source file.
        source_file_name: str = os.path.basename(file_patches_dir).removesuffix("-patches") + ".c"
        source_file: str = f"{build_folder}/{source_file_name}"
        # Iterate through the patches directory for the specific file. The patches directory contain
        # the individual patches.
        for idx, patch_file_name in enumerate(os.listdir(f"./patches/{sub_folder_name}/{file_patches_dir}")):
            patch_file: str = f"./patches/{sub_folder_name}/{file_patches_dir}/{patch_file_name}"
            # Iterate through patch files.
            if os.path.isfile(patch_file) and patch_file.endswith(".patch"):
                result_file: str = f"patched-src/{sub_folder_name}/{source_file_name.removesuffix('.c')}-{idx}.c"
                # print(f"{source_file} + {patch_file} ===> {result_file}\n\n")
                apply_patch(source_file, patch_file, result_file)

### Remove Assert Statements

In [4]:
# Create the patched-src-2 folder in case it does not exist.
if os.path.exists("patched-src-2"):
    print("./patched-src-2 already exists.")
else:
    os.makedirs("patched-src-2")
    print("Created patched-src-2")

for folder in build_folders:
    folder_path: str = f"patched-src-2/{os.path.basename(folder)}"
    if os.path.exists(folder_path):
        print(folder_path, "already exists.")
    else:
        os.makedirs(folder_path)
        print("Created", folder_path)

Created patched-src-2
Created patched-src-2/hopfield_nets
Created patched-src-2/poly_approx
Created patched-src-2/reach_prob_density
Created patched-src-2/reinforcement_learning


In [5]:
def remove_asserts(source_file_path: str, save_file_path: str) -> None:
    """Will remove asserts from source file."""
    # Iterate through patched-src and remove from all files, the assert statements.
    with open(source_file_path, "r") as file:
        content: str = str(file.read())
        has_assert: bool = True
        while has_assert:
            if "assert(" in content:
                # Remove assert by finding the begining and end of assert.
                assert_start_idx: int = content.find("assert(");
                assert_end_idx: int = content.find(";", assert_start_idx);
                content = content[:assert_start_idx] + content[assert_end_idx-1:]
            else:
                has_assert = False
        else:
            # Save
            with open(save_file_path, "w") as save_file:
                save_file.write(content)

In [19]:
for build_folder in build_folders:
    cat_name: str = os.path.basename(build_folder)
    for patched_src_name in os.listdir(f"./patched-src/{cat_name}"):
        patched_src: str = f"./patched-src/{cat_name}/{patched_src_name}"
        save_path: str = f"./patched-src-2/{cat_name}/{patched_src_name}"
        remove_asserts(patched_src, save_path)

KeyboardInterrupt: 